<a href="https://colab.research.google.com/github/LiviaAniely/Aplica-es-do-Processamento-de-Linguagem-Natural/blob/main/LiviaA_KNN_para_Busca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KNN para busca

O KNN é uma técnica de aprendizado de máquina que se baseia na similaridade entre pontos de dados e pode ser uma ferramenta valiosa para a busca e recuperação de informações. Dada uma instância de entrada, o algoritmo K-NN (K-Nearest Neighbors) retorna as K instâncias mais próximas (ou mais similares).
Essa atividade prática foca no uso do algoritmo KNN no contexto de busca em uma coleção de notícias. O objetivo é explorar como o KNN pode ser empregado para realizar buscas por notícias similares à uma dada notícia em uma coleção de notícias obtidas da Folha de São Paulo.

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd 'drive/MyDrive/Cursos UFCG/NLP - 2020.3/Código'

In [ ]:
import pandas as pd
import numpy as np
import nltk

In [ ]:
df = pd.read_csv("https://gist.githubusercontent.com/issilva5/665cb1a2fee6d0e7a144dd1f48fd07d4/raw/ba4b864530493ab1ef8bd59f695d43cba726d3b8/noticias_folha_sample.csv")
df.head()

df['tituloEconteudo'] = df['titulo'] + " "+ df['conteudo'] # cria uma 3° coluna que junta as infomações de titulo e conteudo para fazer a análise
df.head()

,titulo,conteudo,tituloEconteudo
0,Desenhos que icebergs deixam no fundo do mar c...,A imagem acima parece um desenho com giz de ce...,Desenhos que icebergs deixam no fundo do mar c...
1,Bolas de neve gigantes aparecem em praia da Si...,"Moradores do Golfo de Ob, no noroeste da Sibér...",Bolas de neve gigantes aparecem em praia da Si...
2,Projeto de termelétrica no litoral de SP gera ...,O plano de construção de um complexo termelétr...,Projeto de termelétrica no litoral de SP gera ...
3,Cidades com mais mata atlântica estão no litor...,"Ubatuba, Ilhabela e São Sebastião, todas no li...",Cidades com mais mata atlântica estão no litor...
4,Câmara aprova projeto que facilita exploração ...,Após mais de sete meses de impasse e intensas ...,Câmara aprova projeto que facilita exploração ...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import stopwords

nltk.download('stopwords')
pt_sw = stopwords.words('portuguese')

#Algoritmo TF
def buildCountVectorizer(docs):
  vect = CountVectorizer(stop_words = pt_sw)
  matrix = vect.fit_transform(docs)
  return matrix, vect

#Algoritmo TF-IDF
def buildTFIDFVectorizer(docs):
  vect = TfidfVectorizer(stop_words = pt_sw)
  matrix = vect.fit_transform(docs)
  return matrix, vect


def top_words(bow, vect, word):
  idx = df.index[df['titulo'].str.contains(word)]
  row = bow.getrow(idx[0])
  weights = []
  key_list = list(vect.vocabulary_.keys())
  val_list = list(vect.vocabulary_.values())
  for i in row.nonzero()[1]:
    position = val_list.index(i)
    weights.append((key_list[position], row.toarray()[0][i]))
  weights = sorted(weights, key = lambda x:x[1], reverse=True)
  return weights



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.neighbors import NearestNeighbors

#k = quant vizinhos
def getNearestNeighbors(k, vect, matrix, query_list):
  query = vect.transform(query_list)
  neigh = NearestNeighbors(n_neighbors=k)
  neigh.fit(matrix)#calcular os vizinhos mais próximos
  knn = neigh.kneighbors(query,k)
  return knn

In [ ]:
#parâmetros: column: coluna da tabela; type: tipo do algoritmo: td ou tfidf; k:
def build_search_for_docs(column, type, k, query_list):

  if(type.upper() == "TF"):
    bow, vect = buildCountVectorizer(df[column])
  elif(type.upper() in ["TFIDF", "TD-IDF"]):
    bow, vect = buildTFIDFVectorizer(df[column])
  else:
    raise Exception("O tipo informado de algoritmo de busca está errado")

  knn = getNearestNeighbors(k, vect, bow, query_list)

  return bow, vect, knn


## 1.Buscando notícias usando palavras-chave


- lista de palavras usadas na busca: drogas, polícia, Sudeste

OBS: Nas buscas desse colab, o algoritmo binário é o mesmo do TF, devido que só a uma única repetição da palavra.

## 1.1. Realização da busca dos 3 documentos mais similares usando o título das notícias



In [ ]:
#Busca usando o titulo das notícias com o algoritmo TF

bow,vect,result = build_search_for_docs("titulo","tf", 3, ["droga polícia sudeste"])

df.iloc[result[1][0]]

,titulo,conteudo,tituloEconteudo
1183,Ou ou,"RIO DE JANEIRO - Como não me canso de dizer, o...",Ou ou RIO DE JANEIRO - Como não me canso de di...
6322,Eles,Adalberto Oliveira dos Santos. Adão Luiz de A...,Eles Adalberto Oliveira dos Santos. Adão Luiz...
212,Consolo,Diferentemente de necrotério (lugar dos mortos...,Consolo Diferentemente de necrotério (lugar do...


In [ ]:
#Busca usando o titulo das notícias com o algoritmo TF-IDF

bow,vect,result3 = build_search_for_docs("titulo","tfidf", 3,["droga polícia sudeste"])

df.iloc[result3[1][0]]


,titulo,conteudo,tituloEconteudo
1183,Ou ou,"RIO DE JANEIRO - Como não me canso de dizer, o...",Ou ou RIO DE JANEIRO - Como não me canso de di...
6322,Eles,Adalberto Oliveira dos Santos. Adão Luiz de A...,Eles Adalberto Oliveira dos Santos. Adão Luiz...
306,"Droga, fronteiras e modelos",Informa a Reuters que um novo ponto quente de ...,"Droga, fronteiras e modelos Informa a Reuters ..."


### Análise:
Nesse primeiro caso, usando os títulos, vemos que com o TF, tivemos as notícias 1183, 6322 e 212, e com o TF-IDF, tivemos as notícias 1183, 6322 e 306. Nesse sentido, para esse caso, ambos algoritmos reultaram em resultados bastante parecidos, apenas com a 3° notícia sendo diferente.
No entanto, quando analisamos o texto dessas notícias, parece que os temas não convergem para o desejado de fato, tirando a notícia 306 que parece mais adequada à busca.

## 2.1. Realização da busca dos 3 documentos mais similares usando o título e o conteúdo das notícias

In [ ]:
#Busca usando o titulo e o conteúdo das notícias com o algoritmo TF

bow,vect,result2 = build_search_for_docs("tituloEconteudo","tf", 3, ["droga polícia sudeste"])

df.iloc[result2[1][0]]

,titulo,conteudo,tituloEconteudo
4236,Tudo sobre Impeachment,Perguntas e respostas,Tudo sobre Impeachment Perguntas e respostas
4235,Supertrunfo da Liga dos Campeões,Compare o desempenho dos principais jogadores ...,Supertrunfo da Liga dos Campeões Compare o des...
3733,Peça Única: Primeiro Olhar: Diesel Black Gold ...,... Leia post completo no blog,Peça Única: Primeiro Olhar: Diesel Black Gold ...


In [ ]:
#Busca usando o titulo e o conteúdo das notícias com o algoritmo TF-IDF

bow,vect,result4 = build_search_for_docs("tituloEconteudo","tfidf", 3,["droga polícia sudeste"])

df.iloc[result4[1][0]]

,titulo,conteudo,tituloEconteudo
1852,"No Paraná, polícia apreende segunda maior carg...",A PRF (Polícia Rodoviária Federal) apreendeu 8...,"No Paraná, polícia apreende segunda maior carg..."
2295,Ministro do STF defende decisão em que liberou...,O ministro do STF (Supremo Tribunal Federal) L...,Ministro do STF defende decisão em que liberou...
2336,EUA relaxam regras para uso da pílula abortiva...,"A FDA (Food and Drug Administration, agência q...",EUA relaxam regras para uso da pílula abortiva...


### Análise:

Nesse segundo caso, usando o título e conteúdos das notícias, vemos que com o TF, tivemos as notícias 4236, 4235 e 3733, e com o TF-IDF, tivemos as notícias 1852, 2295 e 2336. Nesse sentido, os 2 algoritmos resultaram em respostas de busca bastante diferentes, sem nenhuma convergência.
Quando analisamos o texto dessas notícias, na busca com TF, as notícias resultantes de fato não parecem tem alguma relação com o tema das palavras usadas para a busca. Diferentemente, do TF-IDF, em que as respostas parecem estar mais alinhadas.

## Resultado:

Dentre as combinações entre algoritmos e tipos de dados, os que resultaram em melhores resultados foram com a busca usando TF-IDF com o título e o conteúdo das notícias. Esse resultado, de certa forma, era esperado visto que o TD-IDF considera os pesos das palavras em um texto, assim, os tokens têm uma melhor análise quanto a sua relevância para esse texto, e também usar o título + conteúdo torna-se mais relevante, visto que muitas vezes, o título pode não conseguir abranger de fato qual o assunto do texto.

##  2.Buscandos notícias semelhantes


- Tratamento da notícia que será usada como base para a busca. Para as análises a seguir, usaremos a notícia 7000 como referência.


In [ ]:
ref_news_title = df['titulo'][7000].split(" ")
ref_news_title_and_content = df['tituloEconteudo'][7000].split(" ")

print(ref_news_title)
print(ref_news_title_and_content)


['Temer', 'é', 'Tite', 'e', 'Dilma', 'é', 'Dunga', 'na', 'seleção,', 'compara', 'Jucá']
['Temer', 'é', 'Tite', 'e', 'Dilma', 'é', 'Dunga', 'na', 'seleção,', 'compara', 'Jucá', 'Líder', 'do', 'governo', 'no', 'Senado,', 'Romero', 'Jucá', '(PMDB-RR)', 'comparou', 'Michel', 'Temer', 'ao', 'técnico', 'Tite,', 'que', 'reergueu', 'a', 'seleção', 'brasileira,', 'e', 'Dilma', 'Rousseff', 'ao', 'antecessor', 'dele,', 'Dunga,', 'que', 'foi', 'demitido', 'da', 'equipe', 'com', 'ela', 'fora', 'da', 'zona', 'de', 'classificação', 'nas', 'eliminatórias', 'para', 'a', 'Copa', 'de', '2018.', '', '"Para', 'fazer', 'uma', 'analogia', 'com', 'o', 'futebol,', 'estávamos', 'lá', 'embaixo', 'nas', 'eliminatórias,', 'mudou', 'o', 'técnico,', 'mudou', 'a', 'forma', 'de', 'atuar,', 'e', 'o', 'Brasil', 'hoje', 'com', 'Tite', 'está', 'em', 'primeiro', 'lugar', 'no', 'ranking', 'do', 'futebol",', 'disse', 'nesta', 'sexta-feira', '(21)', 'a', 'uma', 'plateia', 'de', 'empresários.', '', 'Analogia', 'semelhante,', '

## 2.1. Realização da busca dos 3 documentos mais similares usando o título das notícias

In [ ]:
#Busca usando o titulo das notícias com o algoritmo TF

bow,vect,result5 = build_search_for_docs("titulo","tf", 3, ref_news_title)

df.iloc[result5[1][0]]

,titulo,conteudo,tituloEconteudo
462,A ficção de Temer,BRASÍLIA - Michel Temer recorreu a um truque a...,A ficção de Temer BRASÍLIA - Michel Temer reco...
1183,Ou ou,"RIO DE JANEIRO - Como não me canso de dizer, o...",Ou ou RIO DE JANEIRO - Como não me canso de di...
1190,A traição de Temer,"Começo por uma confissão. Por bom tempo, desen...",A traição de Temer Começo por uma confissão. P...


In [ ]:
#Busca usando o titulo das notícias com o algoritmo TF-IDF

bow,vect,result6 = build_search_for_docs("titulo","tfidf", 3, ref_news_title)

df.iloc[result6[1][0]]

,titulo,conteudo,tituloEconteudo
675,Temer versus Temer,A defesa de Michel Temer quer convencer o país...,Temer versus Temer A defesa de Michel Temer qu...
462,A ficção de Temer,BRASÍLIA - Michel Temer recorreu a um truque a...,A ficção de Temer BRASÍLIA - Michel Temer reco...
1183,Ou ou,"RIO DE JANEIRO - Como não me canso de dizer, o...",Ou ou RIO DE JANEIRO - Como não me canso de di...


### Análise:

Nesse primeiro caso, usando os títulos, vemos que com o TF, tivemos as notícias 462, 1183 e 1190, e com o TF-IDF, tivemos as notícias 675, 462 e 1183. Nesse sentido, os resultados são diferentes, apenas com uma notícia em comum.

Quando vemos os temas/conteúdos dessas respostas, eles de fato remetem à notícia referência da busca, com exceção da notícia 1183.


## 2.2. Realização da busca dos 3 documentos mais similares usando o título e o conteúdo das notícias

In [ ]:
#Busca usando o titulo e o conteúdo das notícias com o algoritmo TF

bow,vect,result7 = build_search_for_docs("tituloEconteudo","tf", 3, ref_news_title_and_content)

df.iloc[result7[1][0]]

,titulo,conteudo,tituloEconteudo
4236,Tudo sobre Impeachment,Perguntas e respostas,Tudo sobre Impeachment Perguntas e respostas
4235,Supertrunfo da Liga dos Campeões,Compare o desempenho dos principais jogadores ...,Supertrunfo da Liga dos Campeões Compare o des...
3733,Peça Única: Primeiro Olhar: Diesel Black Gold ...,... Leia post completo no blog,Peça Única: Primeiro Olhar: Diesel Black Gold ...


In [ ]:
#Busca usando o titulo e o conteúdo das notícias com o algoritmo TF-IDF

bow,vect,result8 = build_search_for_docs("tituloEconteudo","tfidf", 3, ref_news_title_and_content)

df.iloc[result8[1][0]]

,titulo,conteudo,tituloEconteudo
6889,Temer traça estratégia de defesa com aliados a...,Às vésperas de embarcar para os Estados Unidos...,Temer traça estratégia de defesa com aliados a...
7046,Grupos que se uniram pela queda de Dilma se se...,"Domingo, 13 de março de 2016. Avenida Paulista...",Grupos que se uniram pela queda de Dilma se se...
4687,Veja o andamento das medidas do governo Temer,"O presidente interino, Michel Temer, anunciou ...",Veja o andamento das medidas do governo Temer ...


### Análise:

Nesse primeiro caso, usando os títulos  e conteúdos, vemos que com o TF, tivemos as notícias 4236, 4235 e 3733, e com o TF-IDF, tivemos as notícias 6889, 7046 e 4687. Nesse sentido, os resultados dos dois algoritmos foram bem diferentes.
Usando o TF, os resultados não têm ligação com a notícia referência. Diferente, do TD-IDF, que a temática converge.

## Resultado:

Dentre as combinações entre algoritmos e tipos de dados, os que resultaram em melhores resultados foram com a busca usando TF-IDF com o título e o conteúdo das notícias. Esse resultado, de certa forma, era esperado visto que o TD-IDF considera os pesos das palavras em um texto, assim, os tokens têm uma melhor análise quanto a sua relevância para esse texto, e também usar o título + conteúdo torna-se mais relevante, visto que muitas vezes, o título pode não conseguir abranger de fato qual o assunto do texto. Também é válido mencionar que usando tanto TF quanto TD-IDF com apenas os títulos, os resultados não foram ruins.

